Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra

from yeast8model import Yeast8Model

# Construct models of cells of interest

In [ ]:
glc_exch_rate = 16.89
wt_ec = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
wt_ec.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt_ec.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

In [ ]:
from yeast8model import biomass_component_list_orig

glucose_bounds = (-4.75, 0) # gives a sensible growth rate for wt
wt_y8 = Yeast8Model("./models/yeast-GEM_8-6-0.xml", growth_id="r_2111", biomass_id="r_4041")
wt_y8.biomass_component_list = biomass_component_list_orig
wt_y8.model.reactions.r_1714.bounds = glucose_bounds
wt_y8.add_media_components(['r_1992'])

# Grid

Note: the grid thing should take ~8 minutes (probably varies: 5-10 minutes).

In [ ]:
from yeast8model import heatmap_ablation_grid, vget_ablation_ratio

Convenience functions

In [ ]:
@np.vectorize
def vget_growthrate(x):
    return x.ablated_flux[0]

In [ ]:
def grid_plots(
    ablation_result_array,
    exch_rate_dict,
    ratio_vmin=0.5,
    ratio_vmax=1.5,
    grid_xlabel_leader='Glucose exchange',
    grid_ylabel_leader='Ammonium exchange',
):
    ratio_array = vget_ablation_ratio(ablation_result_array)
    growthrate_array = vget_growthrate(ablation_result_array)
    ratios = ratio_array[1:,1:].ravel()
    growthrates = growthrate_array[1:,1:].ravel()

    fig, ax = plt.subplots(nrows=3, ncols=1, figsize=(7,18))
    
    xmax = np.max(list(exch_rate_dict.values())[0])
    ymax = np.max(list(exch_rate_dict.values())[0])
    grid_xlabel = f"{grid_xlabel_leader} (% max = {xmax:.2f})"
    grid_ylabel = f"{grid_ylabel_leader} (% max = {ymax:.2f})"

    heatmap_ablation_grid(
        ax[0], exch_rate_dict, ratio_array, percent_saturation=True, vmin=ratio_vmin, vmax=ratio_vmax
    )
    ax[0].set_xlabel(grid_xlabel)
    ax[0].set_ylabel(grid_ylabel)

    heatmap_ablation_grid(
        ax[1], exch_rate_dict, growthrate_array, percent_saturation=True, cbar_label="growth rate"
    )
    ax[1].set_xlabel(grid_xlabel)
    ax[1].set_ylabel(grid_ylabel)

    ax[2].scatter(growthrates, ratios)
    ax[2].set_xlabel('Growth rate (/h)')
    ax[2].set_ylabel('Ablation ratio')

## Enzyme-constrained

`linspace` values are based on saturation exchange rates from the saturation curves.  These values may be different from the optimal uptake values from optimising the unmodified wild-type model.

### Glucose × ammonium

In [ ]:
exch_rate_dict = {
    "r_1714": np.linspace(0, 2*8.45, 16), # glucose
    "r_1654": np.linspace(0, 2*1.45, 16), # ammonium
}
ablation_result_array = wt_ec.ablation_grid(exch_rate_dict)

In [ ]:
grid_plots(ablation_result_array, exch_rate_dict)

### Pyruvate × ammonium

In [ ]:
exch_rate_dict = {
    "r_2033": np.linspace(0, 2*4.27, 16), # pyruvate
    "r_1654": np.linspace(0, 2*1.45, 16), # ammonium
}
ablation_result_array = wt_ec.ablation_grid(exch_rate_dict)

In [ ]:
grid_plots(ablation_result_array, exch_rate_dict, ratio_vmin=0.75, ratio_vmax=1.25, grid_xlabel_leader="Pyruvate exchange")

## Original yeast-GEM (no enzyme constraints)

### Glucose × ammonium

In [ ]:
exch_rate_dict = {
    "r_1714": np.linspace(0, 2*4.75, 16), # glucose
    "r_1654": np.linspace(0, 2*2.88, 16), # ammonium
}
ablation_result_array = wt_y8.ablation_grid(exch_rate_dict)

In [ ]:
grid_plots(ablation_result_array, exch_rate_dict, ratio_vmin=0.75, ratio_vmax=1.25)

### Pyruvate × ammonium

In [ ]:
exch_rate_dict = {
    "r_2033": np.linspace(0, 2*13.32, 16), # pyruvate
    "r_1654": np.linspace(0, 2*2.88, 16), # ammonium
}
ablation_result_array = wt_y8.ablation_grid(exch_rate_dict)

In [ ]:
grid_plots(ablation_result_array, exch_rate_dict, ratio_vmin=0.75, ratio_vmax=1.25, grid_xlabel_leader="Pyruvate exchange")